## Get API key

In [1]:
import os
import getpass

os.environ["Cohere_API_KEY"] = getpass.getpass()

In [2]:
cohere_api_key = os.environ["Cohere_API_KEY"]
print(cohere_api_key)


nt9wg84wNjGfVEwC9ysQuAVdEJlT9bDYjtMeMA8g


## Compatible versions of langchain packages

In [4]:
!pip install --upgrade \
  "pydantic>=2.0" \
  "langchain-core>=0.1.8" \
  "langchain>=0.1.8" \
  "langchain-cohere>=0.0.6" \
  "langchain-huggingface>=0.0.3"  # or latest


  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.2 MB/s eta 0:00:00-:--:--
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.21
    Uninstalling pydantic-1.10.21:
      Successfully uninstalled pydantic-1.10.21
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain-huggingface
    Found existing installation: langchain-huggingface 0.0.3
    Uninstalling langchain-huggingface-0.0.3:
      Successfully uninstalled langchain-huggingface-0.0.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


## Load Cohere Embedding

In [3]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model = "embed-multilingual-light-v3.0",
    max_retries=3,
    request_timeout=20
)

## Load docs for embedding

In [4]:
from langchain_community.document_loaders import WikipediaLoader

wikipedia_docs = WikipediaLoader(query = "فیلم ارباب حلقه‌ها",
                                 load_max_docs = 6,
                                 lang = 'fa').load()

# print(wikipedia_docs.content[0])

In [5]:
print(wikipedia_docs[0].metadata.keys())

dict_keys(['title', 'summary', 'source'])


In [6]:
print(wikipedia_docs[0].metadata["summary"])

ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها نوشتهٔ جی. آر. آر. تالکین به نگارش درآورده‌اند و پس از یاران حلقه (۲۰۰۱) و ارباب حلقه‌ها (۲۰۰۲) آخرین قسمت از سه‌گانهٔ ارباب حلقه‌ها به‌شمار می‌رود. بازگشت پادشاه دنبالهٔ داستان دو برج است و در آن فرودو، سم و گالوم راهی کوه نابودی در موردور می‌شوند تا حلقه یگانه را نابود کنند، اما از اهداف واقعی گالوم اطلاعی ندارند. در همین حین گندالف، آراگورن، لگولاس، گیملی و سایر افراد در حال متحد شدن با یکدیگر علیه سائورون و لژیون‌هایش در میناس تیریت هستند. بازیگران این فیلم الایجا وود، ایان مک‌کلن، لیو تایلر، ویگو مورتنسن، شان آستین، کیت بلانشت، جان ریس-دیویس، برنارد هیل، بیلی بوید، دامینیک مانهن، اورلاندو بلوم، هوگو ویوینگ، میراندا اتو، دیوید ونهام، کارل اربن، جان نوبل، اندی سرکیس، ایان هولم، و شان بین هستند.
تأمین بودجهٔ و توزیع بازگشت پادشاه بر عهده

In [7]:
## remove unnecessary fields, "summary"
for doc in wikipedia_docs:
    doc.metadata.pop("summary", None)

In [8]:
print(wikipedia_docs[0].metadata["summary"])

KeyError: 'summary'

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap = 32,
)

## Split documents into smaller chunks
## for better performance
## and to avoid exceeding token limits
splitted_docs = text_splitter.split_documents(wikipedia_docs)

print(f"Number of splitted documents: {len(splitted_docs)}")

Number of splitted documents: 132


## VectorStore - Chroma

- https://api.python.langchain.com/en/latest/vectorstores/langchain_chroma.vectorstores.Chroma.html

In [15]:
!pip install langchain-chroma

In [12]:
from langchain_chroma import Chroma

vectorStore = Chroma.from_documents(
    documents = splitted_docs,
    embedding=embeddings,
    collection_name = "lord_of_the_rings", # The name of the collection
    persist_directory = "./collections", # Where to save the collection
)

In [13]:
print(f"Total number of documents in the collection: {vectorStore._collection.count()}")

Total number of documents in the collection: 132


## Load from the collection of vectorstore

In [6]:
from langchain_chroma import Chroma

vectorStore = Chroma(
    embedding_function=embeddings,
    collection_name = "lord_of_the_rings",
    persist_directory = "./collections",
)
print(f"Total number of documents in the collection: {vectorStore._collection.count()}")

Total number of documents in the collection: 132


In [7]:
query = 'ارباب حلقه‌های سوم چند جایزه‌ی گلدن گلوب برنده شد؟'

results = vectorStore.similarity_search(query,
                                        k=3,
                                        filter = {'title': 'ارباب حلقه‌ها: بازگشت پادشاه'})

for res in results:
    print(res.metadata["title"])
    print(res.page_content)
    print('-'*100)
    print("\n")

ارباب حلقه‌ها: بازگشت پادشاه
بر پایهٔ جلد سوم ارباب حلقه‌ها نوشتهٔ جی. آر. آر. تالکین به نگارش درآورده‌اند و پس از یاران حلقه (۲۰۰۱) و ارباب حلقه‌ها (۲۰۰۲) آخرین قسمت از سه‌گانهٔ ارباب حلقه‌ها به‌شمار می‌رود. بازگشت پادشاه دنبالهٔ داستان دو برج است و در آن فرودو، سم و گالوم راهی کوه
----------------------------------------------------------------------------------------------------


ارباب حلقه‌ها: بازگشت پادشاه
ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها
----------------------------------------------------------------------------------------------------


ارباب حلقه‌ها: بازگشت پادشاه
را که برای آن‌ها نامزد شده بود، از جمله جایزهٔ بهترین فیلم را کسب کرد. همچنین پس از سه‌گانهٔ اصلی جنگ ستارگان، دومین مجموعه فیلمی بود که همه آثارش برندهٔ جایزهٔ اسکار بهترین جلوه‌های تصویری شدند.
-----

In [18]:
## Test english query
query = 'How many Golden Globe awards did the third Lord of the Rings film win?'

results = vectorStore.similarity_search(query, k = 3,
                                         filter = {'title': 'ارباب حلقه‌ها: بازگشت پادشاه'})

for result in results:
    print('Document Title:', result.metadata['title'])
    print('Document Content:', result.page_content)
    print('-'*100)


Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: را که برای آن‌ها نامزد شده بود، از جمله جایزهٔ بهترین فیلم را کسب کرد. همچنین پس از سه‌گانهٔ اصلی جنگ ستارگان، دومین مجموعه فیلمی بود که همه آثارش برندهٔ جایزهٔ اسکار بهترین جلوه‌های تصویری شدند.
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: مانند دیگر فیلم‌های این سه‌گانه، بازگشت پادشاه به‌طور گسترده‌ای به عنوان یکی از برترین و تاثیرگذارترین فیلم‌های ساخته شده شناخته می‌شود. این فیلم افتخارات متعددی 

## Return similarity score

- Default score is euclidean distance
- so, the less score, the less distance, the higher similarity

In [8]:
results = vectorStore.similarity_search_with_score(
    query,
    k=3,
    filter = {'title': 'ارباب حلقه‌ها: بازگشت پادشاه'},
)
for result, score in results:
    print('Document Title:', result.metadata['title'])
    print('Document Content:', result.page_content)
    print('Score:', score)
    print('-'*100)

Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: بر پایهٔ جلد سوم ارباب حلقه‌ها نوشتهٔ جی. آر. آر. تالکین به نگارش درآورده‌اند و پس از یاران حلقه (۲۰۰۱) و ارباب حلقه‌ها (۲۰۰۲) آخرین قسمت از سه‌گانهٔ ارباب حلقه‌ها به‌شمار می‌رود. بازگشت پادشاه دنبالهٔ داستان دو برج است و در آن فرودو، سم و گالوم راهی کوه
Score: 0.8173982501029968
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها
Score: 0.9185944199562073
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: را که برای آن‌ها نامزد شده بود، از جمله جایزهٔ بهتر

## Cosine similarity

- score range is (0,1)

In [21]:
cosine_vectorStore = Chroma.from_documents(
    documents = splitted_docs,
    embedding=embeddings,
    collection_name = "lord_of_the_rings_cosine",
    persist_directory = "./collections",
    collection_metadata={"hnsw:space": "cosine"}
)

In [22]:
results = cosine_vectorStore.similarity_search_with_score(query,
                                         k=3,
                                         filter = {'title': 'ارباب حلقه‌ها: بازگشت پادشاه'})

for res, score in results:
    print('Document Title:', res.metadata['title'])
    print('Document Content:', res.page_content)
    print('Score:', score)
    print('-'*100)

Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها
Score: 0.5819506645202637
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: را که برای آن‌ها نامزد شده بود، از جمله جایزهٔ بهترین فیلم را کسب کرد. همچنین پس از سه‌گانهٔ اصلی جنگ ستارگان، دومین مجموعه فیلمی بود که همه آثارش برندهٔ جایزهٔ اسکار بهترین جلوه‌های تصویری شدند.
Score: 0.5909125804901123
----------------------------------------------------------------------------------------------------
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: مانند دیگر فیلم‌های این سه‌گانه، بازگشت پادشاه به‌طور گسترده‌ای به عنوان یکی از برترین و تاثیرگذارترین فیلم‌ها

## 0-1 score

In [23]:
query = 'ارباب حلقه‌های سوم چند جایزه‌ی گلدن گلوب برنده شد؟'

results = vectorStore.similarity_search_with_relevance_scores(query,
                                                              k = 3,
                                                              filter = {'title': 'ارباب حلقه‌ها: بازگشت پادشاه'})

for result in results:
    doc = result[0]
    score = result[1]
    print('Relevance Score:', score)
    print('Document Title:', doc.metadata['title'])
    print('Document Content:', doc.page_content)
    print('-'*100)

Relevance Score: 0.42201215442215345
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: بر پایهٔ جلد سوم ارباب حلقه‌ها نوشتهٔ جی. آر. آر. تالکین به نگارش درآورده‌اند و پس از یاران حلقه (۲۰۰۱) و ارباب حلقه‌ها (۲۰۰۲) آخرین قسمت از سه‌گانهٔ ارباب حلقه‌ها به‌شمار می‌رود. بازگشت پادشاه دنبالهٔ داستان دو برج است و در آن فرودو، سم و گالوم راهی کوه
----------------------------------------------------------------------------------------------------
Relevance Score: 0.3504556564888427
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Content: ارباب حلقه‌ها: بازگشت پادشاه (انگلیسی: The Lord of the Rings: The Return of the King) یک فیلم ماجراجویی فانتزی حماسی به کارگردانی پیتر جکسون است که در سال ۲۰۰۳ منتشر شد. فیلم‌نامهٔ فیلم را فرن والش، فیلیپا بوینس و جکسون بر پایهٔ جلد سوم ارباب حلقه‌ها
----------------------------------------------------------------------------------------------------
Relevance Score: 0.3160235778636252
Document Title: ارباب حلقه‌ها: بازگشت پادشاه
Document Con